In [11]:
from pydantic import BaseModel
import pandas as pd

class Input(BaseModel):
    gender: str
    age: int
    height: float
    weight: float
    family_history:str
    FAVC: str
    FCVC: float
    NCP: float
    CAEC: str
    SMOKE: str
    CH2O: float
    SCC: str
    FAF: float
    TUE: float
    CALC: str
    MTRANS: str

input = Input(
    gender= "Female",
  age= 21,
  height= 1.62,
  weight= 64.0,
  family_history= "yes",
  FAVC= "no",
  FCVC= 2.0,
  NCP= 3.0,
  CAEC= "Sometimes",
  SMOKE= "no",
  CH2O= 2.0,
  SCC= "no",
  FAF= 0.0,
  TUE= 1.0,
  CALC= "no",
  MTRANS= "Public_Transportation"
)
df = pd.DataFrame([input.model_dump()])
df


,gender,age,height,weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Female,21,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation
